In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
df = pd.read_csv('data/merged_tables.csv')
df_knmi = pd.read_csv('data/knmi_data.csv')

In [ ]:
print(df.shape)
df.head()

In [ ]:
print(df_knmi.shape)
df_knmi.head()

In [ ]:
# Create a combined Year-Quarter column in the format 'YYYY-QX' and insert it as the first column
df.insert(0, 'Year_Quarter', df['Year'].astype(int).astype(str) + '-Q' + df['Quarter'].astype(int).astype(str))

# Drop the now redundant 'Year' and 'Quarter' columns
df = df.drop(columns=['Year', 'Quarter'])

# Sort the DataFrame by 'BedrijfstakkenBranchesSBI2008' and 'Year_Quarter' for better chronological order
df = df.sort_values(by=['BedrijfstakkenBranchesSBI2008', 'Year_Quarter']).reset_index(drop=True)

# Display the modified DataFrame with the new column and sorting
print(df.shape)
df.head()

In [ ]:
# Create the four new columns by shifting the values of '80072ned_Ziekteverzuimpercentage_1' by 1, 2, 3, and 4 quarters
df['80072ned_Ziekteverzuimpercentage_1_lag_1'] = df.groupby('BedrijfstakkenBranchesSBI2008')['80072ned_Ziekteverzuimpercentage_1'].shift(1)
df['80072ned_Ziekteverzuimpercentage_1_lag_2'] = df.groupby('BedrijfstakkenBranchesSBI2008')['80072ned_Ziekteverzuimpercentage_1'].shift(2)
df['80072ned_Ziekteverzuimpercentage_1_lag_3'] = df.groupby('BedrijfstakkenBranchesSBI2008')['80072ned_Ziekteverzuimpercentage_1'].shift(3)
df['80072ned_Ziekteverzuimpercentage_1_lag_4'] = df.groupby('BedrijfstakkenBranchesSBI2008')['80072ned_Ziekteverzuimpercentage_1'].shift(4)

# Create the four future columns by shifting the values of '80072ned_Ziekteverzuimpercentage_1' backward by 1, 2, 3, and 4 quarters
df['80072ned_Ziekteverzuimpercentage_1_lead_1'] = df.groupby('BedrijfstakkenBranchesSBI2008')['80072ned_Ziekteverzuimpercentage_1'].shift(-1)
df['80072ned_Ziekteverzuimpercentage_1_lead_2'] = df.groupby('BedrijfstakkenBranchesSBI2008')['80072ned_Ziekteverzuimpercentage_1'].shift(-2)
df['80072ned_Ziekteverzuimpercentage_1_lead_3'] = df.groupby('BedrijfstakkenBranchesSBI2008')['80072ned_Ziekteverzuimpercentage_1'].shift(-3)
df['80072ned_Ziekteverzuimpercentage_1_lead_4'] = df.groupby('BedrijfstakkenBranchesSBI2008')['80072ned_Ziekteverzuimpercentage_1'].shift(-4)

# Display the full DataFrame to verify the new columns
print(df.shape)
df.head()

In [ ]:
# Assume df and df_knmi are already loaded as per your examples
# Step 1: Create a DataFrame of unique industries from df
industries = df[['BedrijfstakkenBranchesSBI2008']].drop_duplicates()

# Step 2: Perform a Cartesian product to expand df_knmi across all industries
expanded_knmi = df_knmi.merge(industries, how='cross')

# Step 3: Merge the expanded df_knmi with the main df based on 'Year_Quarter'
df = df.merge(expanded_knmi, on=['Year_Quarter', 'BedrijfstakkenBranchesSBI2008'], how='left')

# Display the result
print(df.shape)
df.head()

In [ ]:
# Define a function to label COVID-19 years
def label_covid_period(row):
    if row['Year_Quarter'].startswith('2020') or row['Year_Quarter'].startswith('2021') or row['Year_Quarter'].startswith('2022'):
        return 1
    else:
        return 0

# Apply the function to create the covid_19 column
df['covid_19'] = df.apply(label_covid_period, axis=1)

# Display the DataFrame to confirm the changes
print(df[['Year_Quarter', 'covid_19']].head())

In [ ]:
df.tail()

In [ ]:
df.info(verbose=True, show_counts=True) #verbose is true because it otherwise doesn't output the entire list of columns

## Correlation

In [ ]:
# Select only numeric columns (float and int types) from the DataFrame
df_numeric = df.select_dtypes(include=['float', 'int'])

# Ensure the target column exists in the filtered numeric DataFrame
if '80072ned_Ziekteverzuimpercentage_1' in df_numeric.columns:
    # Calculate the correlation matrix and extract correlations with the target column
    df_num_corr = df_numeric.corr()['80072ned_Ziekteverzuimpercentage_1']
    
    # Filter for features with strong correlations (absolute correlation > 0.5)
    golden_features_list = df_num_corr[abs(df_num_corr) > 0.5].sort_values(ascending=False)
    
    # Print the results if there are any strongly correlated features
    print("There are {} strongly correlated values with ziekteverzuim:\n{}".format(len(golden_features_list), golden_features_list))
else:
    print("The target column '80072ned_Ziekteverzuimpercentage_1' is not in the numeric columns.")


In [ ]:
# Initialize a dictionary to store correlation results
correlation_results_by_industry = {}

# Define the target variables
target_columns = [
    '80072ned_Ziekteverzuimpercentage_1',
    '80072ned_Ziekteverzuimpercentage_1_lead_1',
    '80072ned_Ziekteverzuimpercentage_1_lead_2',
    '80072ned_Ziekteverzuimpercentage_1_lead_3',
    '80072ned_Ziekteverzuimpercentage_1_lead_4'
]

# Set correlation threshold
correlation_threshold = 0.7

# Group by industry
for industry, group_df in df.groupby('BedrijfstakkenBranchesSBI2008'):
    # Select numeric columns
    group_df_numeric = group_df.select_dtypes(include=[float, int])
    
    # Initialize a dictionary to store correlation results for this industry
    industry_correlation_results = {}
    
    for target in target_columns:
        # Check if the target exists in the group
        if target in group_df_numeric.columns:
            # Calculate correlation matrix
            corr_matrix = group_df_numeric.corr()
            target_corr = corr_matrix[target]
            
            # Retain features with strong correlation
            strongly_correlated_features = target_corr[abs(target_corr) > correlation_threshold].sort_values(ascending=False)
            
            # Store the results
            industry_correlation_results[target] = strongly_correlated_features
    
    # Store results for this industry
    if industry_correlation_results:
        correlation_results_by_industry[industry] = industry_correlation_results

# Output the results
print("Correlation Analysis Results:")
for industry, results in correlation_results_by_industry.items():
    print(f"Industry: {industry}")
    for target, correlations in results.items():
        print(f"  Target: {target}")
        print(f"  Strong correlations:\n{correlations}\n")


In [ ]:
correlation_results_by_industry

In [ ]:
import os

# Create a directory to store the datasets if it doesn't exist
output_folder = "data"
os.makedirs(output_folder, exist_ok=True)

# Define the target variables
target_columns = [
    '80072ned_Ziekteverzuimpercentage_1',
    '80072ned_Ziekteverzuimpercentage_1_lead_1',
    '80072ned_Ziekteverzuimpercentage_1_lead_2',
    '80072ned_Ziekteverzuimpercentage_1_lead_3',
    '80072ned_Ziekteverzuimpercentage_1_lead_4'
]

# Filter the data for 'C Industrie'
industry = 'C Industrie'
industry_data = df[df['BedrijfstakkenBranchesSBI2008'] == industry]

# Process each target variable
for target in target_columns:
    # Check if the target has correlation results for this industry
    if industry in correlation_results_by_industry and target in correlation_results_by_industry[industry]:
        # Get the features strongly correlated with the current target
        correlated_features = correlation_results_by_industry[industry][target].index.tolist()
        
        # Add the target variable itself
        correlated_features.append(target)
        
        # Add 'Year_Quarter' as it should always be included
        correlated_features.append('Year_Quarter')
        
        # Ensure only valid columns are selected (features that exist in the DataFrame)
        valid_columns = [col for col in correlated_features if col in industry_data.columns]
        
        # Filter the data to include only strongly correlated features
        filtered_data = industry_data[valid_columns]
        
        # Remove duplicate column names (if any)
        filtered_data = filtered_data.loc[:, ~filtered_data.columns.duplicated()]
        
        # Save the filtered dataset as a CSV file
        output_path = os.path.join(output_folder, f"{target}.csv")
        filtered_data.to_csv(output_path, index=False)
        print(f"Dataset for {target} (C Industrie) saved to {output_path}.")


In [ ]:
# Access the index (column names) from the correlation Series
column_names = golden_features_list.index.tolist()

# Get the column names that meet the VIF and lag condition
vif_filtered_columns = vif_filtered['Feature'].tolist()

# Combine correlation-filtered and VIF-filtered columns
selected_columns = list(set(column_names) & set(vif_filtered_columns))

# Add additional necessary columns
additional_columns = ['Year_Quarter', 'BedrijfstakkenBranchesSBI2008']
all_columns = selected_columns + additional_columns

# Filter the DataFrame
df_filtered = df[all_columns]

# Display the filtered DataFrame
df_filtered.head()


In [ ]:
df_final = df_filtered.dropna()
print(df_final.shape)
df_final.head()

In [ ]:
df_final.to_json('data/df_final.json')

## Ziekteverzuim

In [ ]:
# Print the description of the Ziekteverzuimpercentage_1 column
print(df['80072ned_Ziekteverzuimpercentage_1'].describe())

# Plot the distribution with seaborn
plt.figure(figsize=(9, 8))
sns.displot(df['80072ned_Ziekteverzuimpercentage_1'], color='g', bins=10, kde=True)
plt.title('Distribution of Ziekteverzuimpercentage_1')
plt.xlabel('Ziekteverzuimpercentage_1')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Plot the data
plt.figure(figsize=(14, 8))

# Boxplot
sns.boxplot(data=df, x='Year_Quarter', y='80072ned_Ziekteverzuimpercentage_1', color='lightblue')

# Adding labels and title
plt.title('Distribution of Ziekteverzuimpercentage_1 by Year-Quarter with Median Trend')
plt.xlabel('Year-Quarter')
plt.ylabel('Ziekteverzuimpercentage_1')

# Rotate x-axis labels for better readability
plt.xticks(rotation=90)

# Show the plot
plt.tight_layout()
plt.legend()
plt.show()

## Numerical data distribution

In [ ]:
list(set(df.dtypes.tolist()))

In [ ]:
df_num = df.select_dtypes(include = ['float64', 'int64'])
df_num.head()

In [ ]:
# Calculate number of rows and columns for subplots
num_cols = 3  # Number of columns in the grid
num_rows = (len(df_num.columns) - 1) // num_cols + 1  # Number of rows needed

# Set up the figure and axes
fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, num_rows * 4))
axes = axes.flatten()  # Flatten the 2D array of axes for easier indexing

# Plot each numeric column
for i, col in enumerate(df_num.columns):
    ax = axes[i]
    sns.histplot(df_num[col], bins=50, kde=True, ax=ax)
    ax.set_title(col)
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
for i in range(0, len(df_num.columns), 5):
    sns.pairplot(data=df_num,
                x_vars=df_num.columns[i:i+5],
                y_vars=['80072ned_Ziekteverzuimpercentage_1'])

In [ ]:
# Define the columns for which we want lagged correlations
columns_to_lag = [
    'airpressure', 'maximum_temperatures', 'mean_temperatures', 
    'minimum_temperatures', 'precipitation', 'covid_19'
]

# Create lagged variables for each column in `columns_to_lag`
for col in columns_to_lag:
    df[f'{col}_lag_1'] = df[col].shift(1)
    df[f'{col}_lag_2'] = df[col].shift(2)

# Recompute numeric-only DataFrame to include new lagged columns
df_numeric = df.select_dtypes(include=[float, int])

# Calculate correlations for the primary column of interest and its lags
correlation_columns = columns_to_lag + [f"{col}_lag_1" for col in columns_to_lag] + [f"{col}_lag_2" for col in columns_to_lag]
correlations = df_numeric.corr()['80072ned_Ziekteverzuimpercentage_1'][correlation_columns]

print("Correlations with 80072ned_Ziekteverzuimpercentage_1:")
print(correlations)

In [ ]:
# Set VIF threshold
vif_threshold = 10

# Initialize a dictionary to store VIF results
vif_results_by_industry = {}

# Function to iteratively remove features with infinite VIF
def calculate_vif_with_removal(features_df):
    features_df = features_df.copy()
    while True:
        vif_data = pd.DataFrame()
        vif_data['Feature'] = features_df.columns
        vif_data['VIF'] = [
            variance_inflation_factor(features_df.values, i) for i in range(features_df.shape[1])
        ]
        
        # Check for infinite VIF values
        max_vif = vif_data['VIF'].max()
        if max_vif == float('inf'):
            # Remove the feature with the highest VIF
            feature_to_remove = vif_data.loc[vif_data['VIF'].idxmax(), 'Feature']
            print(f"Removing feature with infinite VIF: {feature_to_remove}")
            features_df = features_df.drop(columns=[feature_to_remove])
        else:
            break
    
    return vif_data

# Iterate over industries identified in correlation analysis
for industry, correlation_results in correlation_results_by_industry.items():
    # Initialize a dictionary to store VIF results for this industry
    industry_vif_results = {}
    
    for target, correlations in correlation_results.items():
        # Extract the strongly correlated features
        correlated_features = correlations.index.tolist()
        
        # Prepare data for VIF calculation
        features_for_vif = df[df['BedrijfstakkenBranchesSBI2008'] == industry][correlated_features]
        
        # Handle missing or infinite values
        features_for_vif = features_for_vif.replace([np.inf, -np.inf], np.nan).dropna()

        if not features_for_vif.empty and features_for_vif.shape[1] > 1:  # Ensure there are enough features
            # Calculate VIF with removal of problematic features
            vif_data = calculate_vif_with_removal(features_for_vif)
            
            # Retain features with VIF <= threshold or containing 'lag'
            selected_features = vif_data[
                (vif_data['VIF'] <= vif_threshold) | (vif_data['Feature'].str.contains('lag'))
            ]
            
            # Store the results
            industry_vif_results[target] = selected_features
    
    # Store VIF results for this industry
    if industry_vif_results:
        vif_results_by_industry[industry] = industry_vif_results

# Output the results
print("VIF Analysis Results:")
for industry, results in vif_results_by_industry.items():
    print(f"Industry: {industry}")
    for target, vif_data in results.items():
        print(f"  Target: {target}")
        print(f"  VIF Data:\n{vif_data}\n")
